In [1]:
import duckdb as ddb
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec

data = ddb.read_parquet("all_reviews/all_reviews.parquet")

In [ ]:
# columns and types
for x, item in enumerate(data.columns):
    print(str(item) + ":  " + str(data.dtypes[x]))

game:  VARCHAR
author_steamid:  BIGINT
author_num_games_owned:  INTEGER
author_num_reviews:  INTEGER
author_playtime_forever:  INTEGER
author_playtime_last_two_weeks:  INTEGER
author_playtime_at_review:  BIGINT
author_last_played:  INTEGER
language:  VARCHAR
review:  VARCHAR
timestamp_created:  INTEGER
timestamp_updated:  INTEGER
voted_up:  INTEGER
votes_up:  INTEGER
votes_funny:  INTEGER
weighted_vote_score:  FLOAT
comment_count:  INTEGER
steam_purchase:  INTEGER
received_for_free:  INTEGER
written_during_early_access:  INTEGER


In [ ]:
# print(data.describe())

['game', 'author_steamid', 'author_num_games_owned', 'author_num_reviews', 'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access']
┌─────────┬─────────────────────────────┬───────────────────────┬────────────────────────┬────────────────────┬─────────────────────────┬────────────────────────────────┬───────────────────────────┬────────────────────┬────────────┬───────────────────────────────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────────────┐
│  aggr   │            game             │    author_steamid     │ author_num_games_owned │ author

In [ ]:
# IDEA:
# review scores from those who got the game for free vs those who had to pay for it
# is there an average of a higher rating for free / not purchased? for what games?

# plan: 
#   gather the top 3 games the most amount of reviews
#   find out the percentage of positive vs negative reviews
#   then find out the percentages from those who puchased vs did not purchase the game (include those who got the game for free)
#   analyze which games had which results, and do some further research on why this may be
#       this could be finding the reviews with the highest helpfulness or most upvotes
#       this could also include doing some outside reseach on reddit / internet

# find games with middle of the road reviews, grab the top 3 in review count
mid_reviewed_games = ddb.sql(f"""
    SELECT
        game,
        COUNT(*) AS review_count,
        AVG(CASE WHEN voted_up = 1 THEN 1.0 ELSE 0 END) * 100 AS average_positive_percentage
    FROM data
    GROUP BY game
    HAVING 
        average_positive_percentage BETWEEN 40 AND 60
        AND review_count > 100
    ORDER BY review_count DESC
    -- LIMIT 10
""").fetchdf()

print(mid_reviewed_games)

                                                   game  review_count  \
0                                         Call of Duty®        451506   
1                                     Battlefield™ 2042        227879   
2                                 鬼谷八荒 Tale of Immortal        219306   
3                                    EA SPORTS™ FIFA 23        141619   
4                            Warhammer 40,000: Darktide         77394   
5                                     雀魂麻将(MahjongSoul)         47821   
6                                              PAYDAY 3         42514   
7                                     theHunter Classic         40053   
8                                              NBA 2K22         38302   
9                                       Bless Unleashed         30108   
10                                          BLOCKADE 3D         26158   
11                       Call of Duty: Infinite Warfare         25907   
12                   Infestation: Survivor Stories 

In [69]:
# figure out purchased percentages
purchased_review_percentages = ddb.sql(f"""
    SELECT
        game,
        COUNT(*) AS review_count,
        SUM(CASE WHEN voted_up = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS positive_review_percentage
    FROM data
    WHERE 
        game IN (SELECT game FROM mid_reviewed_games)
        AND steam_purchase = 1
        AND received_for_free != 1
    GROUP BY game
    HAVING review_count > 100
    ORDER BY review_count DESC
""").fetchdf()

print(purchased_review_percentages)

# figure out not percentages
not_purchased_review_percentages = ddb.sql(f"""
    SELECT
        game,
        COUNT(*) AS review_count,
        SUM(CASE WHEN voted_up = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS positive_review_percentage
    FROM data
    WHERE 
        game IN (SELECT game FROM mid_reviewed_games)
        AND (steam_purchase = 0 OR received_for_free = 1)
    GROUP BY game
    HAVING review_count > 100
    ORDER BY review_count DESC
""").fetchdf()

print(purchased_review_percentages)
print(not_purchased_review_percentages)

                                                  game  review_count  \
0                                        Call of Duty®        244781   
1                                鬼谷八荒 Tale of Immortal        185173   
2                                    Battlefield™ 2042        169993   
3                                   EA SPORTS™ FIFA 23        113922   
4                           Warhammer 40,000: Darktide         65316   
5                                             PAYDAY 3         32143   
6                                             NBA 2K22         29107   
7                              Wo Long: Fallen Dynasty         17532   
8                   Infestation: Survivor Stories 2020         17236   
9                              Need for Speed™ Unbound         15935   
10                                            Devotion         15703   
11                                           Wayfinder         15373   
12                      Monster Hunter World: Iceborne         1

In [71]:
# join table and find out the percentage difference for each game
joined_percentages = ddb.sql(f"""
    SELECT
        p.game,
        p.review_count,
        n.positive_review_percentage,
        p.positive_review_percentage,
        (n.positive_review_percentage - p.positive_review_percentage) AS percentage_difference
    FROM purchased_review_percentages p
    JOIN not_purchased_review_percentages n ON p.game = n.game
    ORDER BY p.review_count DESC;
""").fetchdf()

print(joined_percentages)

                                                  game  review_count  \
0                                        Call of Duty®        244781   
1                                鬼谷八荒 Tale of Immortal        185173   
2                                    Battlefield™ 2042        169993   
3                                   EA SPORTS™ FIFA 23        113922   
4                           Warhammer 40,000: Darktide         65316   
5                                             PAYDAY 3         32143   
6                                             NBA 2K22         29107   
7                              Wo Long: Fallen Dynasty         17532   
8                   Infestation: Survivor Stories 2020         17236   
9                              Need for Speed™ Unbound         15935   
10                                            Devotion         15703   
11                                           Wayfinder         15373   
12                      Monster Hunter World: Iceborne         1

In [72]:
# calculate average (>50000 reviews)
average_percentage_50000 = ddb.sql(f"""
    SELECT AVG(percentage_difference) AS average_percentage_difference_50000
    FROM joined_percentages
    WHERE review_count > 50000;
""").fetchdf()

# calculate average (>10000 reviews)
average_percentage_10000 = ddb.sql(f"""
    SELECT AVG(percentage_difference) AS average_percentage_difference_10000
    FROM joined_percentages
    WHERE review_count > 10000;
""").fetchdf()

# calculate average (>1000 reviews)
average_percentage_1000 = ddb.sql(f"""
    SELECT AVG(percentage_difference) AS average_percentage_difference_1000
    FROM joined_percentages
    WHERE review_count > 1000;
""").fetchdf()

# calculate max
max_percentage = ddb.sql(f"""
    SELECT game, review_count, percentage_difference
    FROM joined_percentages
    -- WHERE review_count > 1000
    ORDER BY percentage_difference DESC
    LIMIT 3;
""").fetchdf()

# calculate min
min_percentage = ddb.sql(f"""
    SELECT game, review_count, percentage_difference
    FROM joined_percentages
    -- WHERE review_count > 1000
    ORDER BY percentage_difference ASC
    LIMIT 3;
""").fetchdf()

# top 10 differences from top reviews
top_10_percentage = ddb.sql(f"""
    SELECT game, review_count, percentage_difference
    FROM joined_percentages
    ORDER BY review_count DESC
    LIMIT 10;
""").fetchdf()

print(average_percentage_50000)
print(average_percentage_10000)
print(average_percentage_1000)
print(max_percentage)
print(min_percentage)
print(top_10_percentage)

   average_percentage_difference_50000
0                            -2.390343
   average_percentage_difference_10000
0                            -2.925769
   average_percentage_difference_1000
0                           -2.574916
                                        game  review_count  \
0  Sid Meier’s Civilization® VI: Leader Pass           392   
1      Airport Firefighters - The Simulation           107   
2     Company of Heroes 2 - Ardennes Assault           266   

   percentage_difference  
0              30.959453  
1              23.720893  
2              23.427967  
                    game  review_count  percentage_difference
0          The Anacrusis           457             -42.429116
1                Grapple           227             -38.770448
2  Capcom Arcade Stadium           294             -34.293878
                                 game  review_count  percentage_difference
0                       Call of Duty®        244781               7.039485
1            